<a href="https://colab.research.google.com/github/Sagi1977/CycleTrading-/blob/StockAnalysis/StockAnalysis_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
STOCK ANALYSIS GUI - GOOGLE COLAB VERSION
Copy this entire code into a Google Colab cell and run it!
"""

# Install required libraries
!pip install yfinance scipy -q

import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
from google.colab import files
import time
import json
from scipy.signal import argrelextrema

class StockAnalysisGUI:
    def __init__(self):
        self.groups = {}  # {group_name: [stock_tickers]}
        self.current_group = None
        self.config_file = 'stock_groups_config.json'
        self.load_config()
        self.setup_ui()

    def update_stock_selection_multi(self, change):
        """Update stock list when groups are selected"""
        if change['new']:
            # Collect all stocks from selected groups
            all_stocks = []
            for group_name in change['new']:
                all_stocks.extend(self.groups.get(group_name, []))
            # Remove duplicates while preserving order
            seen = set()
            unique_stocks = []
            for stock in all_stocks:
                if stock not in seen:
                    seen.add(stock)
                    unique_stocks.append(stock)
            self.stock_selection.options = unique_stocks

    def select_all_stocks(self, b):
        """Select all stocks from selected groups"""
        selected_groups = self.analysis_group_selection.value
        if selected_groups:
            all_stocks = []
            for group_name in selected_groups:
                all_stocks.extend(self.groups.get(group_name, []))
            # Remove duplicates
            unique_stocks = list(set(all_stocks))
            self.stock_selection.value = tuple(unique_stocks)

    def save_config(self):
        """Save groups and stocks to JSON file"""
        try:
            with open(self.config_file, 'w') as f:
                json.dump(self.groups, f, indent=2)
            return True
        except Exception as e:
            print(f"Error saving config: {str(e)}")
            return False

    def load_config(self):
        """Load groups and stocks from JSON file"""
        try:
            with open(self.config_file, 'r') as f:
                self.groups = json.load(f)
            print(f"✅ Loaded {len(self.groups)} groups from saved configuration")
        except FileNotFoundError:
            print("ℹ️ No saved configuration found. Starting fresh.")
            self.groups = {}
        except Exception as e:
            print(f"⚠️ Error loading config: {str(e)}")
            self.groups = {}

    def download_config(self, b):
        """Download configuration file"""
        if self.save_config():
            files.download(self.config_file)
            with self.output:
                clear_output()
                display(HTML(f"""
                <div style='background: #d1fae5; padding: 15px; border-radius: 10px; border-left: 5px solid #10b981;'>
                    <p style='color: #065f46; margin: 0;'>✅ Configuration file downloaded: {self.config_file}</p>
                </div>
                """))

    def upload_config(self, b):
        """Upload configuration file"""
        with self.output:
            clear_output()
            display(HTML("""
            <div style='background: #dbeafe; padding: 15px; border-radius: 10px; border-left: 5px solid #3b82f6;'>
                <p style='color: #1e40af; margin: 0;'>📤 Please select your configuration file to upload...</p>
            </div>
            """))

        uploaded = files.upload()

        if uploaded:
            filename = list(uploaded.keys())[0]
            try:
                with open(filename, 'r') as f:
                    self.groups = json.load(f)

                # Update UI
                group_list = list(self.groups.keys())
                self.group_dropdown.options = group_list
                self.analysis_group_selection.options = group_list

                with self.output:
                    clear_output()
                    display(HTML(f"""
                    <div style='background: #d1fae5; padding: 15px; border-radius: 10px; border-left: 5px solid #10b981;'>
                        <p style='color: #065f46; margin: 0;'>✅ Configuration loaded! {len(self.groups)} groups imported.</p>
                    </div>
                    """))

                self.update_groups_display()

            except Exception as e:
                with self.output:
                    clear_output()
                    display(HTML(f"""
                    <div style='background: #fee2e2; padding: 15px; border-radius: 10px; border-left: 5px solid #ef4444;'>
                        <p style='color: #dc2626; margin: 0;'>❌ Error loading file: {str(e)}</p>
                    </div>
                    """))

    def setup_ui(self):
        # Title
        display(HTML("""
        <div style='background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
                    padding: 30px; border-radius: 15px; text-align: center; margin-bottom: 20px;'>
            <h1 style='color: white; margin: 0; font-size: 36px;'>📊 Stock Analysis GUI</h1>
            <p style='color: #e0e7ff; margin: 10px 0 0 0; font-size: 16px;'>
                Create groups, add stocks, and analyze market data
            </p>
        </div>
        """))

        # Section 1: Group Management
        display(HTML("<h2 style='color: #667eea; margin-top: 30px;'>1️⃣ Group Management</h2>"))

        self.group_name_input = widgets.Text(
            placeholder='Enter group name (e.g., ETFs)',
            description='Group Name:',
            style={'description_width': '120px'},
            layout=widgets.Layout(width='400px')
        )

        self.add_group_button = widgets.Button(
            description='➕ Add Group',
            button_style='success',
            layout=widgets.Layout(width='150px')
        )
        self.add_group_button.on_click(self.add_group)

        display(widgets.HBox([self.group_name_input, self.add_group_button]))

        # Save/Load configuration buttons
        self.save_config_button = widgets.Button(
            description='💾 Save Config',
            button_style='info',
            layout=widgets.Layout(width='150px'),
            tooltip='Download your groups & stocks configuration'
        )
        self.save_config_button.on_click(self.download_config)

        self.load_config_button = widgets.Button(
            description='📤 Load Config',
            button_style='warning',
            layout=widgets.Layout(width='150px'),
            tooltip='Upload a saved configuration file'
        )
        self.load_config_button.on_click(self.upload_config)

        display(widgets.HBox([self.save_config_button, self.load_config_button]))

        # Section 2: Stock Management
        display(HTML("<h2 style='color: #667eea; margin-top: 30px;'>2️⃣ Stock Management</h2>"))

        self.group_dropdown = widgets.Dropdown(
            options=[],
            description='Select Group:',
            style={'description_width': '120px'},
            layout=widgets.Layout(width='400px')
        )

        self.stock_ticker_input = widgets.Text(
            placeholder='Enter ticker (e.g., SPY)',
            description='Stock Ticker:',
            style={'description_width': '120px'},
            layout=widgets.Layout(width='400px')
        )

        self.add_stock_button = widgets.Button(
            description='➕ Add Stock',
            button_style='info',
            layout=widgets.Layout(width='150px')
        )
        self.add_stock_button.on_click(self.add_stock)

        display(self.group_dropdown)
        display(widgets.HBox([self.stock_ticker_input, self.add_stock_button]))

        # Section 3: Stock Selection
        display(HTML("<h2 style='color: #667eea; margin-top: 30px;'>3️⃣ Select Stocks for Analysis</h2>"))

        self.analysis_group_selection = widgets.SelectMultiple(
            options=[],
            description='Select Groups:',
            style={'description_width': '120px'},
            layout=widgets.Layout(width='400px', height='100px')
        )
        self.analysis_group_selection.observe(self.update_stock_selection_multi, names='value')

        self.stock_selection = widgets.SelectMultiple(
            options=[],
            description='Stocks:',
            style={'description_width': '120px'},
            layout=widgets.Layout(width='400px', height='150px')
        )

        self.select_all_button = widgets.Button(
            description='✅ Select ALL',
            button_style='warning',
            layout=widgets.Layout(width='150px')
        )
        self.select_all_button.on_click(self.select_all_stocks)

        display(self.analysis_group_selection)
        display(self.stock_selection)
        display(self.select_all_button)

        # Section 4: Period Selection
        display(HTML("<h2 style='color: #667eea; margin-top: 30px;'>4️⃣ Select Period</h2>"))

        self.period_dropdown = widgets.Dropdown(
            options=[20, 50, 100, 150, 200],
            value=200,
            description='Period (days):',
            style={'description_width': '120px'},
            layout=widgets.Layout(width='400px')
        )

        display(self.period_dropdown)

        # Output area for messages
        self.output = widgets.Output()
        display(self.output)

        # Section 5: Run Analysis
        display(HTML("<h2 style='color: #667eea; margin-top: 30px;'>5️⃣ Run Analysis</h2>"))

        self.run_analysis_button = widgets.Button(
            description='🚀 Run Analysis',
            button_style='danger',
            layout=widgets.Layout(width='200px', height='50px'),
            style={'font_weight': 'bold'},
            tooltip='Analyze selected stocks only'
        )
        self.run_analysis_button.on_click(self.run_analysis)

        self.run_full_analysis_button = widgets.Button(
            description='🚀 Run FULL Analysis',
            button_style='success',
            layout=widgets.Layout(width='200px', height='50px'),
            style={'font_weight': 'bold'},
            tooltip='Analyze ALL stocks from ALL groups'
        )
        self.run_full_analysis_button.on_click(self.run_full_analysis)

        display(widgets.HBox([self.run_analysis_button, self.run_full_analysis_button]))

        # Analysis output area
        self.analysis_output = widgets.Output()
        display(self.analysis_output)

        # Groups display area
        display(HTML("<hr style='border: 2px solid #667eea; margin: 40px 0;'>"))
        self.groups_display = widgets.Output()
        display(self.groups_display)

        self.update_groups_display()

    def add_group(self, b):
        try:
            group_name = self.group_name_input.value.strip()

            if not group_name:
                with self.output:
                    clear_output()
                    display(HTML("""
                    <div style='background: #fee2e2; padding: 15px; border-radius: 10px; border-left: 5px solid #ef4444;'>
                        <p style='color: #dc2626; margin: 0;'>❌ Please enter a group name!</p>
                    </div>
                    """))
                return

            if group_name in self.groups:
                with self.output:
                    clear_output()
                    display(HTML(f"""
                    <div style='background: #fef3c7; padding: 15px; border-radius: 10px; border-left: 5px solid #f59e0b;'>
                        <p style='color: #92400e; margin: 0;'>⚠️ Group "{group_name}" already exists!</p>
                    </div>
                    """))
                return

            self.groups[group_name] = []
            self.group_name_input.value = ''

            # Update dropdowns
            group_list = list(self.groups.keys())
            self.group_dropdown.options = group_list
            self.analysis_group_selection.options = group_list

            with self.output:
                clear_output()
                display(HTML(f"""
                <div style='background: #d1fae5; padding: 15px; border-radius: 10px; border-left: 5px solid #10b981;'>
                    <p style='color: #065f46; margin: 0;'>✅ Group "{group_name}" added successfully!</p>
                </div>
                """))

            self.update_groups_display()

            # Auto-save after adding group
            self.save_config()

        except Exception as e:
            with self.output:
                clear_output()
                display(HTML(f"""
                <div style='background: #fee2e2; padding: 15px; border-radius: 10px; border-left: 5px solid #ef4444;'>
                    <p style='color: #dc2626; margin: 0;'>❌ Error adding group: {str(e)}</p>
                </div>
                """))

    def add_stock(self, b):
        try:
            selected_group = self.group_dropdown.value
            ticker = self.stock_ticker_input.value.strip().upper()

            if not selected_group:
                with self.output:
                    clear_output()
                    display(HTML("""
                    <div style='background: #fee2e2; padding: 15px; border-radius: 10px; border-left: 5px solid #ef4444;'>
                        <p style='color: #dc2626; margin: 0;'>❌ Please select a group first!</p>
                    </div>
                    """))
                return

            if not ticker:
                with self.output:
                    clear_output()
                    display(HTML("""
                    <div style='background: #fee2e2; padding: 15px; border-radius: 10px; border-left: 5px solid #ef4444;'>
                        <p style='color: #dc2626; margin: 0;'>❌ Please enter a stock ticker!</p>
                    </div>
                    """))
                return

            if ticker in self.groups[selected_group]:
                with self.output:
                    clear_output()
                    display(HTML(f"""
                    <div style='background: #fef3c7; padding: 15px; border-radius: 10px; border-left: 5px solid #f59e0b;'>
                        <p style='color: #92400e; margin: 0;'>⚠️ {ticker} already exists in {selected_group}!</p>
                    </div>
                    """))
                return

            self.groups[selected_group].append(ticker)
            self.stock_ticker_input.value = ''

            # Update stock selection dropdown if the same group is selected
            if selected_group in self.analysis_group_selection.value:
                self.update_stock_selection_multi({'new': self.analysis_group_selection.value})

            with self.output:
                clear_output()
                display(HTML(f"""
                <div style='background: #d1fae5; padding: 15px; border-radius: 10px; border-left: 5px solid #10b981;'>
                    <p style='color: #065f46; margin: 0;'>✅ {ticker} added to {selected_group}!</p>
                </div>
                """))

            self.update_groups_display()

            # Auto-save after adding stock
            self.save_config()

        except Exception as e:
            with self.output:
                clear_output()
                display(HTML(f"""
                <div style='background: #fee2e2; padding: 15px; border-radius: 10px; border-left: 5px solid #ef4444;'>
                    <p style='color: #dc2626; margin: 0;'>❌ Error adding stock: {str(e)}</p>
                </div>
                """))

    def update_groups_display(self):
        try:
            with self.groups_display:
                clear_output()
                if self.groups:
                    display(HTML("<h3 style='color: #667eea; margin-top: 30px;'>📋 Current Groups & Stocks</h3>"))

                    for group_name, stocks in self.groups.items():
                        stocks_list = ', '.join(stocks) if stocks else '<em>No stocks added</em>'
                        display(HTML(f"""
                        <div style='background: #f8fafc; padding: 15px; border-radius: 10px;
                                    border-left: 5px solid #667eea; margin-bottom: 10px;'>
                            <h4 style='color: #1e293b; margin: 0 0 10px 0;'>{group_name}</h4>
                            <p style='color: #64748b; margin: 0;'>{stocks_list}</p>
                        </div>
                        """))
        except Exception as e:
            print(f"Error updating display: {str(e)}")

    def run_analysis(self, b):
        with self.analysis_output:
            clear_output()

            try:
                # Validate inputs
                selected_groups = list(self.analysis_group_selection.value)
                selected_stocks = list(self.stock_selection.value)
                period = self.period_dropdown.value

                print("DEBUG: Selected groups:", selected_groups)
                print("DEBUG: Selected stocks:", selected_stocks)
                print("DEBUG: Period:", period)
                print("=" * 70)

                if not selected_groups:
                    display(HTML("""
                    <div style='background: #fee2e2; padding: 15px; border-radius: 10px; border-left: 5px solid #ef4444;'>
                        <p style='color: #dc2626; margin: 0;'>❌ Please select at least one group!</p>
                    </div>
                    """))
                    return

                if not selected_stocks:
                    display(HTML("""
                    <div style='background: #fee2e2; padding: 15px; border-radius: 10px; border-left: 5px solid #ef4444;'>
                        <p style='color: #dc2626; margin: 0;'>❌ Please select at least one stock!</p>
                    </div>
                    """))
                    return

                display(HTML("""
                <div style='background: #dbeafe; padding: 20px; border-radius: 10px; border-left: 5px solid #3b82f6;'>
                    <h3 style='color: #1e40af; margin-top: 0;'>🚀 Running Analysis...</h3>
                </div>
                """))

                print(f"📊 Groups: {', '.join(selected_groups)}")
                print(f"📈 Stocks: {', '.join(selected_stocks)}")
                print(f"📅 Period: {period} trading days")
                print("=" * 70 + "\n")

                # Calculate date range - need extra days for MA300 calculation
                end_date = datetime.now()
                start_date = end_date - timedelta(days=int((period + 300) * 1.5))  # Extra buffer for MA300

                all_data = []

                for idx, ticker in enumerate(selected_stocks, 1):
                    print(f"[{idx}/{len(selected_stocks)}] Fetching {ticker}...", end=" ")

                    try:
                        stock = yf.Ticker(ticker)
                        df = stock.history(start=start_date, end=end_date)

                        if len(df) == 0:
                            print("❌ No data available")
                            continue

                        # Calculate indicators first (need more data)
                        # Then get exact number of trading days requested at the end

                        # Validate data
                        if len(df) < period * 0.9:  # Allow 10% tolerance
                            print(f"⚠️ Only {len(df)} days available (expected {period})")

                        # Format data
                        df = df.reset_index()
                        df['Ticker'] = ticker

                        # Find which group this ticker belongs to
                        ticker_group = None
                        for grp in selected_groups:
                            if ticker in self.groups.get(grp, []):
                                ticker_group = grp
                                break

                        df['Group'] = ticker_group if ticker_group else 'Unknown'

                        # Calculate Technical Indicators
                        # 1. RSI (14)
                        delta = df['Close'].diff()
                        gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
                        loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
                        rs = gain / loss
                        df['RSI_14'] = 100 - (100 / (1 + rs))

                        # 2. MACD Histogram
                        ema12 = df['Close'].ewm(span=12, adjust=False).mean()
                        ema26 = df['Close'].ewm(span=26, adjust=False).mean()
                        macd_line = ema12 - ema26
                        signal_line = macd_line.ewm(span=9, adjust=False).mean()
                        df['MACD_Hist'] = macd_line - signal_line

                        # 3. ATR %14 (ATR(14)/Close)
                        high_low = df['High'] - df['Low']
                        high_close = abs(df['High'] - df['Close'].shift())
                        low_close = abs(df['Low'] - df['Close'].shift())
                        true_range = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
                        atr = true_range.rolling(window=14).mean()
                        df['ATR_pct_14'] = (atr / df['Close']) * 100

                        # 4. Moving Averages (50, 150, 200, 300)
                        df['MA50'] = df['Close'].rolling(window=50).mean()
                        df['MA150'] = df['Close'].rolling(window=150).mean()
                        df['MA200'] = df['Close'].rolling(window=200).mean()
                        df['MA300'] = df['Close'].rolling(window=300).mean()

                        # 5. Volume (20-day average)
                        df['Vol_Avg_20'] = df['Volume'].rolling(window=20).mean()

                        # 6. Fibonacci Retracement Levels
                        # Determine trend direction (last 45 days for daily)
                        recent_45 = df['Close'].tail(45)
                        if len(recent_45) >= 45:
                            trend_start = recent_45.iloc[0]
                            trend_end = recent_45.iloc[-1]
                            is_uptrend = trend_end > trend_start

                            # Find swing high and low in last 45 days
                            swing_high = recent_45.max()
                            swing_low = recent_45.min()

                            if is_uptrend:
                                # Uptrend: calculate from low to high
                                fib_range = swing_high - swing_low
                                df['Fib_Daily_0'] = swing_low
                                df['Fib_Daily_23.6'] = swing_low + (fib_range * 0.236)
                                df['Fib_Daily_38.2'] = swing_low + (fib_range * 0.382)
                                df['Fib_Daily_50'] = swing_low + (fib_range * 0.5)
                                df['Fib_Daily_61.8'] = swing_low + (fib_range * 0.618)
                                df['Fib_Daily_78.6'] = swing_low + (fib_range * 0.786)
                                df['Fib_Daily_100'] = swing_high
                                df['Trend_Daily'] = 'Uptrend'
                            else:
                                # Downtrend: calculate from high to low
                                fib_range = swing_high - swing_low
                                df['Fib_Daily_0'] = swing_high
                                df['Fib_Daily_23.6'] = swing_high - (fib_range * 0.236)
                                df['Fib_Daily_38.2'] = swing_high - (fib_range * 0.382)
                                df['Fib_Daily_50'] = swing_high - (fib_range * 0.5)
                                df['Fib_Daily_61.8'] = swing_high - (fib_range * 0.618)
                                df['Fib_Daily_78.6'] = swing_high - (fib_range * 0.786)
                                df['Fib_Daily_100'] = swing_low
                                df['Trend_Daily'] = 'Downtrend'
                        else:
                            # Not enough data for daily fibonacci
                            df['Fib_Daily_0'] = None
                            df['Fib_Daily_23.6'] = None
                            df['Fib_Daily_38.2'] = None
                            df['Fib_Daily_50'] = None
                            df['Fib_Daily_61.8'] = None
                            df['Fib_Daily_78.6'] = None
                            df['Fib_Daily_100'] = None
                            df['Trend_Daily'] = 'Unknown'

                        # Weekly Fibonacci (resample to weekly and look at last ~13 weeks)
                        df_copy = df.copy()
                        df_copy['Date_dt'] = pd.to_datetime(df_copy['Date'])
                        df_copy.set_index('Date_dt', inplace=True)
                        weekly_data = df_copy['Close'].resample('W-FRI').last().dropna()

                        if len(weekly_data) >= 13:
                            recent_weekly = weekly_data.tail(13)
                            weekly_trend_start = recent_weekly.iloc[0]
                            weekly_trend_end = recent_weekly.iloc[-1]
                            is_weekly_uptrend = weekly_trend_end > weekly_trend_start

                            weekly_swing_high = recent_weekly.max()
                            weekly_swing_low = recent_weekly.min()

                            if is_weekly_uptrend:
                                weekly_fib_range = weekly_swing_high - weekly_swing_low
                                df['Fib_Weekly_0'] = weekly_swing_low
                                df['Fib_Weekly_23.6'] = weekly_swing_low + (weekly_fib_range * 0.236)
                                df['Fib_Weekly_38.2'] = weekly_swing_low + (weekly_fib_range * 0.382)
                                df['Fib_Weekly_50'] = weekly_swing_low + (weekly_fib_range * 0.5)
                                df['Fib_Weekly_61.8'] = weekly_swing_low + (weekly_fib_range * 0.618)
                                df['Fib_Weekly_78.6'] = weekly_swing_low + (weekly_fib_range * 0.786)
                                df['Fib_Weekly_100'] = weekly_swing_high
                                df['Trend_Weekly'] = 'Uptrend'
                            else:
                                weekly_fib_range = weekly_swing_high - weekly_swing_low
                                df['Fib_Weekly_0'] = weekly_swing_high
                                df['Fib_Weekly_23.6'] = weekly_swing_high - (weekly_fib_range * 0.236)
                                df['Fib_Weekly_38.2'] = weekly_swing_high - (weekly_fib_range * 0.382)
                                df['Fib_Weekly_50'] = weekly_swing_high - (weekly_fib_range * 0.5)
                                df['Fib_Weekly_61.8'] = weekly_swing_high - (weekly_fib_range * 0.618)
                                df['Fib_Weekly_78.6'] = weekly_swing_high - (weekly_fib_range * 0.786)
                                df['Fib_Weekly_100'] = weekly_swing_low
                                df['Trend_Weekly'] = 'Downtrend'
                        else:
                            df['Fib_Weekly_0'] = None
                            df['Fib_Weekly_23.6'] = None
                            df['Fib_Weekly_38.2'] = None
                            df['Fib_Weekly_50'] = None
                            df['Fib_Weekly_61.8'] = None
                            df['Fib_Weekly_78.6'] = None
                            df['Fib_Weekly_100'] = None
                            df['Trend_Weekly'] = 'Unknown'

                        # Golden Pocket Detection (61.8% ±0.3%)
                        current_price = df['Close'].iloc[-1]

                        # Daily Golden Pocket
                        if pd.notna(df['Fib_Daily_61.8'].iloc[-1]):
                            daily_618 = df['Fib_Daily_61.8'].iloc[-1]
                            golden_pocket_range = daily_618 * 0.003  # ±0.3%
                            df['Golden_Pocket_Daily'] = (current_price >= daily_618 - golden_pocket_range) & (current_price <= daily_618 + golden_pocket_range)
                        else:
                            df['Golden_Pocket_Daily'] = False

                        # Weekly Golden Pocket
                        if pd.notna(df['Fib_Weekly_61.8'].iloc[-1]):
                            weekly_618 = df['Fib_Weekly_61.8'].iloc[-1]
                            golden_pocket_range = weekly_618 * 0.003  # ±0.3%
                            df['Golden_Pocket_Weekly'] = (current_price >= weekly_618 - golden_pocket_range) & (current_price <= weekly_618 + golden_pocket_range)
                        else:
                            df['Golden_Pocket_Weekly'] = False

                        # Confluence Detection (Daily+Weekly 61.8% align within ±0.5%)
                        if pd.notna(df['Fib_Daily_61.8'].iloc[-1]) and pd.notna(df['Fib_Weekly_61.8'].iloc[-1]):
                            daily_618 = df['Fib_Daily_61.8'].iloc[-1]
                            weekly_618 = df['Fib_Weekly_61.8'].iloc[-1]
                            confluence_tolerance = daily_618 * 0.005  # ±0.5%
                            df['Confluence_61.8'] = abs(daily_618 - weekly_618) <= confluence_tolerance
                        else:
                            df['Confluence_61.8'] = False

                        # 7. ADX (Average Directional Index) - Trend Strength
                        high_low = df['High'] - df['Low']
                        high_close = abs(df['High'] - df['Close'].shift())
                        low_close = abs(df['Low'] - df['Close'].shift())
                        true_range = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)

                        plus_dm = df['High'].diff()
                        minus_dm = -df['Low'].diff()
                        plus_dm[plus_dm < 0] = 0
                        minus_dm[minus_dm < 0] = 0

                        tr14 = true_range.rolling(window=14).sum()
                        plus_di14 = 100 * (plus_dm.rolling(window=14).sum() / tr14)
                        minus_di14 = 100 * (minus_dm.rolling(window=14).sum() / tr14)

                        dx = 100 * abs(plus_di14 - minus_di14) / (plus_di14 + minus_di14)
                        df['ADX'] = dx.rolling(window=14).mean()

                        # 8. Stochastic Oscillator
                        low_14 = df['Low'].rolling(window=14).min()
                        high_14 = df['High'].rolling(window=14).max()
                        df['Stoch_K'] = 100 * ((df['Close'] - low_14) / (high_14 - low_14))
                        df['Stoch_D'] = df['Stoch_K'].rolling(window=3).mean()

                        # 9. Bollinger Bands
                        bb_period = 20
                        bb_std = 2
                        df['BB_Middle'] = df['Close'].rolling(window=bb_period).mean()
                        rolling_std = df['Close'].rolling(window=bb_period).std()
                        df['BB_Upper'] = df['BB_Middle'] + (rolling_std * bb_std)
                        df['BB_Lower'] = df['BB_Middle'] - (rolling_std * bb_std)
                        df['BB_Width'] = ((df['BB_Upper'] - df['BB_Lower']) / df['BB_Middle']) * 100

                        # 10. Support/Resistance Levels (using swing highs/lows from last 50 days)
                        lookback = min(50, len(df))
                        recent_data = df['Close'].tail(lookback)

                        # Find local maxima and minima
                        from scipy.signal import argrelextrema
                        if len(recent_data) > 10:
                            high_indices = argrelextrema(recent_data.values, np.greater, order=5)[0]
                            low_indices = argrelextrema(recent_data.values, np.less, order=5)[0]

                            resistance_levels = recent_data.iloc[high_indices].values if len(high_indices) > 0 else []
                            support_levels = recent_data.iloc[low_indices].values if len(low_indices) > 0 else []

                            # Get closest levels
                            current_price = df['Close'].iloc[-1]

                            if len(resistance_levels) > 0:
                                resistance_above = [r for r in resistance_levels if r > current_price]
                                df['Resistance_1'] = min(resistance_above) if resistance_above else None
                            else:
                                df['Resistance_1'] = None

                            if len(support_levels) > 0:
                                support_below = [s for s in support_levels if s < current_price]
                                df['Support_1'] = max(support_below) if support_below else None
                            else:
                                df['Support_1'] = None
                        else:
                            df['Resistance_1'] = None
                            df['Support_1'] = None

                        # 11. Swing High/Low Detection
                        swing_window = 5
                        df['Swing_High'] = False
                        df['Swing_Low'] = False
                        df['Swing_High_Price'] = None
                        df['Swing_Low_Price'] = None

                        for i in range(swing_window, len(df) - swing_window):
                            # Check if it's a swing high
                            if df['High'].iloc[i] == df['High'].iloc[i-swing_window:i+swing_window+1].max():
                                df.loc[df.index[i], 'Swing_High'] = True
                                df.loc[df.index[i], 'Swing_High_Price'] = df['High'].iloc[i]

                            # Check if it's a swing low
                            if df['Low'].iloc[i] == df['Low'].iloc[i-swing_window:i+swing_window+1].min():
                                df.loc[df.index[i], 'Swing_Low'] = True
                                df.loc[df.index[i], 'Swing_Low_Price'] = df['Low'].iloc[i]

                        # Select columns in desired order
                        df = df[['Date', 'Group', 'Ticker', 'Open', 'High', 'Low', 'Close', 'Volume',
                                'RSI_14', 'MACD_Hist', 'ATR_pct_14', 'MA50', 'MA150', 'MA200', 'MA300', 'Vol_Avg_20',
                                'ADX', 'Stoch_K', 'Stoch_D',
                                'BB_Upper', 'BB_Middle', 'BB_Lower', 'BB_Width',
                                'Support_1', 'Resistance_1',
                                'Swing_High', 'Swing_Low', 'Swing_High_Price', 'Swing_Low_Price',
                                'Trend_Daily', 'Fib_Daily_0', 'Fib_Daily_23.6', 'Fib_Daily_38.2', 'Fib_Daily_50',
                                'Fib_Daily_61.8', 'Fib_Daily_78.6', 'Fib_Daily_100',
                                'Trend_Weekly', 'Fib_Weekly_0', 'Fib_Weekly_23.6', 'Fib_Weekly_38.2', 'Fib_Weekly_50',
                                'Fib_Weekly_61.8', 'Fib_Weekly_78.6', 'Fib_Weekly_100',
                                'Golden_Pocket_Daily', 'Golden_Pocket_Weekly', 'Confluence_61.8']]
                        df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')

                        # Get exact number of trading days requested (after calculations)
                        df = df.tail(period)

                        # Validate data
                        if len(df) < period * 0.9:  # Allow 10% tolerance
                            print(f"⚠️ Only {len(df)} days available (expected {period})")

                        all_data.append(df)

                        latest_price = df['Close'].iloc[-1]
                        print(f"✅ {len(df)} days | ${latest_price:.2f}")

                        time.sleep(0.3)

                    except Exception as e:
                        print(f"❌ Error: {str(e)}")

                # Export to CSV
                if all_data:
                    print("\n" + "=" * 70)
                    print("📊 GENERATING CSV...")

                    final_data = pd.concat(all_data, ignore_index=True)
                    final_data = final_data.sort_values(['Date', 'Ticker'])

                    # Generate filename
                    now = datetime.now()
                    filename = f"StockAnalysis_{now.strftime('%d_%m_%Y_%H_%M')}.csv"

                    final_data.to_csv(filename, index=False)

                    print(f"✅ Total rows: {len(final_data)}")
                    print(f"✅ File created: {filename}")
                    print("=" * 70)

                    # Display preview
                    display(HTML("<h3 style='color: #10b981; margin-top: 20px;'>✅ Analysis Complete!</h3>"))
                    display(HTML(f"<p style='color: #64748b;'>📁 File: <strong>{filename}</strong></p>"))

                    display(HTML("<h4 style='color: #3b82f6;'>📋 Data Preview (First 10 rows):</h4>"))
                    display(final_data.head(10))

                    # Download file
                    files.download(filename)

                    display(HTML(f"""
                    <div style='background: #d1fae5; padding: 20px; border-radius: 10px;
                                border-left: 5px solid #10b981; margin-top: 20px;'>
                        <h3 style='color: #065f46; margin-top: 0;'>✅ Success!</h3>
                        <p style='color: #047857; margin: 0;'>
                            <strong>{filename}</strong> has been downloaded!<br>
                            Total rows: {len(final_data)} | Stocks: {len(selected_stocks)} | Period: {period} days
                        </p>
                    </div>
                    """))
                else:
                    display(HTML("""
                    <div style='background: #fee2e2; padding: 20px; border-radius: 10px;
                                border-left: 5px solid #ef4444; margin-top: 20px;'>
                        <h3 style='color: #991b1b; margin-top: 0;'>❌ Error</h3>
                        <p style='color: #dc2626; margin: 0;'>
                            No data was fetched. Please check tickers and try again.
                        </p>
                    </div>
                    """))

            except Exception as e:
                print("\n" + "=" * 70)
                print(f"CRITICAL ERROR: {str(e)}")
                print("=" * 70)
                import traceback
                traceback.print_exc()

                display(HTML(f"""
                <div style='background: #fee2e2; padding: 20px; border-radius: 10px;
                            border-left: 5px solid #ef4444; margin-top: 20px;'>
                    <h3 style='color: #991b1b; margin-top: 0;'>❌ Critical Error</h3>
                    <p style='color: #dc2626; margin: 0;'>
                        {str(e)}
                    </p>
                </div>
                """))

    def run_full_analysis(self, b):
        """Run analysis on ALL stocks from ALL groups"""
        with self.analysis_output:
            clear_output()

            try:
                # Get all stocks from all groups
                if not self.groups:
                    display(HTML("""
                    <div style='background: #fee2e2; padding: 15px; border-radius: 10px; border-left: 5px solid #ef4444;'>
                        <p style='color: #dc2626; margin: 0;'>❌ No groups found! Please add groups and stocks first.</p>
                    </div>
                    """))
                    return

                all_stocks = []
                all_groups = []
                for group_name, stocks in self.groups.items():
                    all_stocks.extend(stocks)
                    all_groups.append(group_name)

                if not all_stocks:
                    display(HTML("""
                    <div style='background: #fee2e2; padding: 15px; border-radius: 10px; border-left: 5px solid #ef4444;'>
                        <p style='color: #dc2626; margin: 0;'>❌ No stocks found! Please add stocks to your groups.</p>
                    </div>
                    """))
                    return

                # Remove duplicates
                unique_stocks = list(set(all_stocks))
                period = self.period_dropdown.value

                display(HTML("""
                <div style='background: #dbeafe; padding: 20px; border-radius: 10px; border-left: 5px solid #3b82f6;'>
                    <h3 style='color: #1e40af; margin-top: 0;'>🚀 Running FULL Analysis...</h3>
                </div>
                """))

                print(f"📊 Groups: {', '.join(all_groups)}")
                print(f"📈 Total Stocks: {len(unique_stocks)} ({', '.join(unique_stocks)})")
                print(f"📅 Period: {period} trading days")
                print("=" * 70 + "\n")

                # Calculate date range - need extra days for MA300 calculation
                end_date = datetime.now()
                start_date = end_date - timedelta(days=int((period + 300) * 1.5))

                all_data = []

                for idx, ticker in enumerate(unique_stocks, 1):
                    print(f"[{idx}/{len(unique_stocks)}] Fetching {ticker}...", end=" ")

                    try:
                        stock = yf.Ticker(ticker)
                        df = stock.history(start=start_date, end=end_date)

                        if len(df) == 0:
                            print("❌ No data available")
                            continue

                        # Calculate indicators first (need more data)
                        # Then get exact number of trading days requested at the end

                        # Validate data
                        if len(df) < period * 0.9:  # Allow 10% tolerance
                            print(f"⚠️ Only {len(df)} days available (expected {period})")

                        # Format data
                        df = df.reset_index()
                        df['Ticker'] = ticker

                        # Find which group this ticker belongs to
                        ticker_group = None
                        for grp, stocks in self.groups.items():
                            if ticker in stocks:
                                ticker_group = grp
                                break

                        df['Group'] = ticker_group if ticker_group else 'Unknown'

                        # Calculate Technical Indicators
                        # 1. RSI (14)
                        delta = df['Close'].diff()
                        gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
                        loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
                        rs = gain / loss
                        df['RSI_14'] = 100 - (100 / (1 + rs))

                        # 2. MACD Histogram
                        ema12 = df['Close'].ewm(span=12, adjust=False).mean()
                        ema26 = df['Close'].ewm(span=26, adjust=False).mean()
                        macd_line = ema12 - ema26
                        signal_line = macd_line.ewm(span=9, adjust=False).mean()
                        df['MACD_Hist'] = macd_line - signal_line

                        # 3. ATR %14 (ATR(14)/Close)
                        high_low = df['High'] - df['Low']
                        high_close = abs(df['High'] - df['Close'].shift())
                        low_close = abs(df['Low'] - df['Close'].shift())
                        true_range = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
                        atr = true_range.rolling(window=14).mean()
                        df['ATR_pct_14'] = (atr / df['Close']) * 100

                        # 4. Moving Averages (50, 150, 200, 300)
                        df['MA50'] = df['Close'].rolling(window=50).mean()
                        df['MA150'] = df['Close'].rolling(window=150).mean()
                        df['MA200'] = df['Close'].rolling(window=200).mean()
                        df['MA300'] = df['Close'].rolling(window=300).mean()

                        # 5. Volume (20-day average)
                        df['Vol_Avg_20'] = df['Volume'].rolling(window=20).mean()

                        # 6. Fibonacci Retracement Levels
                        # Determine trend direction (last 45 days for daily)
                        recent_45 = df['Close'].tail(45)
                        if len(recent_45) >= 45:
                            trend_start = recent_45.iloc[0]
                            trend_end = recent_45.iloc[-1]
                            is_uptrend = trend_end > trend_start

                            # Find swing high and low in last 45 days
                            swing_high = recent_45.max()
                            swing_low = recent_45.min()

                            if is_uptrend:
                                # Uptrend: calculate from low to high
                                fib_range = swing_high - swing_low
                                df['Fib_Daily_0'] = swing_low
                                df['Fib_Daily_23.6'] = swing_low + (fib_range * 0.236)
                                df['Fib_Daily_38.2'] = swing_low + (fib_range * 0.382)
                                df['Fib_Daily_50'] = swing_low + (fib_range * 0.5)
                                df['Fib_Daily_61.8'] = swing_low + (fib_range * 0.618)
                                df['Fib_Daily_78.6'] = swing_low + (fib_range * 0.786)
                                df['Fib_Daily_100'] = swing_high
                                df['Trend_Daily'] = 'Uptrend'
                            else:
                                # Downtrend: calculate from high to low
                                fib_range = swing_high - swing_low
                                df['Fib_Daily_0'] = swing_high
                                df['Fib_Daily_23.6'] = swing_high - (fib_range * 0.236)
                                df['Fib_Daily_38.2'] = swing_high - (fib_range * 0.382)
                                df['Fib_Daily_50'] = swing_high - (fib_range * 0.5)
                                df['Fib_Daily_61.8'] = swing_high - (fib_range * 0.618)
                                df['Fib_Daily_78.6'] = swing_high - (fib_range * 0.786)
                                df['Fib_Daily_100'] = swing_low
                                df['Trend_Daily'] = 'Downtrend'
                        else:
                            # Not enough data for daily fibonacci
                            df['Fib_Daily_0'] = None
                            df['Fib_Daily_23.6'] = None
                            df['Fib_Daily_38.2'] = None
                            df['Fib_Daily_50'] = None
                            df['Fib_Daily_61.8'] = None
                            df['Fib_Daily_78.6'] = None
                            df['Fib_Daily_100'] = None
                            df['Trend_Daily'] = 'Unknown'

                        # Weekly Fibonacci (resample to weekly and look at last ~13 weeks)
                        df_copy = df.copy()
                        df_copy['Date_dt'] = pd.to_datetime(df_copy['Date'])
                        df_copy.set_index('Date_dt', inplace=True)
                        weekly_data = df_copy['Close'].resample('W-FRI').last().dropna()

                        if len(weekly_data) >= 13:
                            recent_weekly = weekly_data.tail(13)
                            weekly_trend_start = recent_weekly.iloc[0]
                            weekly_trend_end = recent_weekly.iloc[-1]
                            is_weekly_uptrend = weekly_trend_end > weekly_trend_start

                            weekly_swing_high = recent_weekly.max()
                            weekly_swing_low = recent_weekly.min()

                            if is_weekly_uptrend:
                                weekly_fib_range = weekly_swing_high - weekly_swing_low
                                df['Fib_Weekly_0'] = weekly_swing_low
                                df['Fib_Weekly_23.6'] = weekly_swing_low + (weekly_fib_range * 0.236)
                                df['Fib_Weekly_38.2'] = weekly_swing_low + (weekly_fib_range * 0.382)
                                df['Fib_Weekly_50'] = weekly_swing_low + (weekly_fib_range * 0.5)
                                df['Fib_Weekly_61.8'] = weekly_swing_low + (weekly_fib_range * 0.618)
                                df['Fib_Weekly_78.6'] = weekly_swing_low + (weekly_fib_range * 0.786)
                                df['Fib_Weekly_100'] = weekly_swing_high
                                df['Trend_Weekly'] = 'Uptrend'
                            else:
                                weekly_fib_range = weekly_swing_high - weekly_swing_low
                                df['Fib_Weekly_0'] = weekly_swing_high
                                df['Fib_Weekly_23.6'] = weekly_swing_high - (weekly_fib_range * 0.236)
                                df['Fib_Weekly_38.2'] = weekly_swing_high - (weekly_fib_range * 0.382)
                                df['Fib_Weekly_50'] = weekly_swing_high - (weekly_fib_range * 0.5)
                                df['Fib_Weekly_61.8'] = weekly_swing_high - (weekly_fib_range * 0.618)
                                df['Fib_Weekly_78.6'] = weekly_swing_high - (weekly_fib_range * 0.786)
                                df['Fib_Weekly_100'] = weekly_swing_low
                                df['Trend_Weekly'] = 'Downtrend'
                        else:
                            df['Fib_Weekly_0'] = None
                            df['Fib_Weekly_23.6'] = None
                            df['Fib_Weekly_38.2'] = None
                            df['Fib_Weekly_50'] = None
                            df['Fib_Weekly_61.8'] = None
                            df['Fib_Weekly_78.6'] = None
                            df['Fib_Weekly_100'] = None
                            df['Trend_Weekly'] = 'Unknown'

                        # Golden Pocket Detection (61.8% ±0.3%)
                        current_price = df['Close'].iloc[-1]

                        # Daily Golden Pocket
                        if pd.notna(df['Fib_Daily_61.8'].iloc[-1]):
                            daily_618 = df['Fib_Daily_61.8'].iloc[-1]
                            golden_pocket_range = daily_618 * 0.003  # ±0.3%
                            df['Golden_Pocket_Daily'] = (current_price >= daily_618 - golden_pocket_range) & (current_price <= daily_618 + golden_pocket_range)
                        else:
                            df['Golden_Pocket_Daily'] = False

                        # Weekly Golden Pocket
                        if pd.notna(df['Fib_Weekly_61.8'].iloc[-1]):
                            weekly_618 = df['Fib_Weekly_61.8'].iloc[-1]
                            golden_pocket_range = weekly_618 * 0.003  # ±0.3%
                            df['Golden_Pocket_Weekly'] = (current_price >= weekly_618 - golden_pocket_range) & (current_price <= weekly_618 + golden_pocket_range)
                        else:
                            df['Golden_Pocket_Weekly'] = False

                        # Confluence Detection (Daily+Weekly 61.8% align within ±0.5%)
                        if pd.notna(df['Fib_Daily_61.8'].iloc[-1]) and pd.notna(df['Fib_Weekly_61.8'].iloc[-1]):
                            daily_618 = df['Fib_Daily_61.8'].iloc[-1]
                            weekly_618 = df['Fib_Weekly_61.8'].iloc[-1]
                            confluence_tolerance = daily_618 * 0.005  # ±0.5%
                            df['Confluence_61.8'] = abs(daily_618 - weekly_618) <= confluence_tolerance
                        else:
                            df['Confluence_61.8'] = False

                        # 7. ADX (Average Directional Index) - Trend Strength
                        high_low = df['High'] - df['Low']
                        high_close = abs(df['High'] - df['Close'].shift())
                        low_close = abs(df['Low'] - df['Close'].shift())
                        true_range = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)

                        plus_dm = df['High'].diff()
                        minus_dm = -df['Low'].diff()
                        plus_dm[plus_dm < 0] = 0
                        minus_dm[minus_dm < 0] = 0

                        tr14 = true_range.rolling(window=14).sum()
                        plus_di14 = 100 * (plus_dm.rolling(window=14).sum() / tr14)
                        minus_di14 = 100 * (minus_dm.rolling(window=14).sum() / tr14)

                        dx = 100 * abs(plus_di14 - minus_di14) / (plus_di14 + minus_di14)
                        df['ADX'] = dx.rolling(window=14).mean()

                        # 8. Stochastic Oscillator
                        low_14 = df['Low'].rolling(window=14).min()
                        high_14 = df['High'].rolling(window=14).max()
                        df['Stoch_K'] = 100 * ((df['Close'] - low_14) / (high_14 - low_14))
                        df['Stoch_D'] = df['Stoch_K'].rolling(window=3).mean()

                        # 9. Bollinger Bands
                        bb_period = 20
                        bb_std = 2
                        df['BB_Middle'] = df['Close'].rolling(window=bb_period).mean()
                        rolling_std = df['Close'].rolling(window=bb_period).std()
                        df['BB_Upper'] = df['BB_Middle'] + (rolling_std * bb_std)
                        df['BB_Lower'] = df['BB_Middle'] - (rolling_std * bb_std)
                        df['BB_Width'] = ((df['BB_Upper'] - df['BB_Lower']) / df['BB_Middle']) * 100

                        # 10. Support/Resistance Levels (using swing highs/lows from last 50 days)
                        lookback = min(50, len(df))
                        recent_data = df['Close'].tail(lookback)

                        # Find local maxima and minima
                        if len(recent_data) > 10:
                            high_indices = argrelextrema(recent_data.values, np.greater, order=5)[0]
                            low_indices = argrelextrema(recent_data.values, np.less, order=5)[0]

                            resistance_levels = recent_data.iloc[high_indices].values if len(high_indices) > 0 else []
                            support_levels = recent_data.iloc[low_indices].values if len(low_indices) > 0 else []

                            # Get closest levels
                            current_price = df['Close'].iloc[-1]

                            if len(resistance_levels) > 0:
                                resistance_above = [r for r in resistance_levels if r > current_price]
                                df['Resistance_1'] = min(resistance_above) if resistance_above else None
                            else:
                                df['Resistance_1'] = None

                            if len(support_levels) > 0:
                                support_below = [s for s in support_levels if s < current_price]
                                df['Support_1'] = max(support_below) if support_below else None
                            else:
                                df['Support_1'] = None
                        else:
                            df['Resistance_1'] = None
                            df['Support_1'] = None

                        # 11. Swing High/Low Detection
                        swing_window = 5
                        df['Swing_High'] = False
                        df['Swing_Low'] = False
                        df['Swing_High_Price'] = None
                        df['Swing_Low_Price'] = None

                        for i in range(swing_window, len(df) - swing_window):
                            # Check if it's a swing high
                            if df['High'].iloc[i] == df['High'].iloc[i-swing_window:i+swing_window+1].max():
                                df.loc[df.index[i], 'Swing_High'] = True
                                df.loc[df.index[i], 'Swing_High_Price'] = df['High'].iloc[i]

                            # Check if it's a swing low
                            if df['Low'].iloc[i] == df['Low'].iloc[i-swing_window:i+swing_window+1].min():
                                df.loc[df.index[i], 'Swing_Low'] = True
                                df.loc[df.index[i], 'Swing_Low_Price'] = df['Low'].iloc[i]

                        # Select columns in desired order
                        df = df[['Date', 'Group', 'Ticker', 'Open', 'High', 'Low', 'Close', 'Volume',
                                'RSI_14', 'MACD_Hist', 'ATR_pct_14', 'MA50', 'MA150', 'MA200', 'MA300', 'Vol_Avg_20',
                                'ADX', 'Stoch_K', 'Stoch_D',
                                'BB_Upper', 'BB_Middle', 'BB_Lower', 'BB_Width',
                                'Support_1', 'Resistance_1',
                                'Swing_High', 'Swing_Low', 'Swing_High_Price', 'Swing_Low_Price',
                                'Trend_Daily', 'Fib_Daily_0', 'Fib_Daily_23.6', 'Fib_Daily_38.2', 'Fib_Daily_50',
                                'Fib_Daily_61.8', 'Fib_Daily_78.6', 'Fib_Daily_100',
                                'Trend_Weekly', 'Fib_Weekly_0', 'Fib_Weekly_23.6', 'Fib_Weekly_38.2', 'Fib_Weekly_50',
                                'Fib_Weekly_61.8', 'Fib_Weekly_78.6', 'Fib_Weekly_100',
                                'Golden_Pocket_Daily', 'Golden_Pocket_Weekly', 'Confluence_61.8']]
                        df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')

                        # Get exact number of trading days requested (after calculations)
                        df = df.tail(period)

                        all_data.append(df)

                        latest_price = df['Close'].iloc[-1]
                        print(f"✅ {len(df)} days | ${latest_price:.2f}")

                        time.sleep(0.3)

                    except Exception as e:
                        print(f"❌ Error: {str(e)}")

                # Export to CSV
                if all_data:
                    print("\n" + "=" * 70)
                    print("📊 GENERATING FULL CSV...")

                    final_data = pd.concat(all_data, ignore_index=True)
                    final_data = final_data.sort_values(['Date', 'Ticker'])

                    # Generate filename
                    now = datetime.now()
                    filename = f"StockAnalysis_FULL_{now.strftime('%d_%m_%Y_%H_%M')}.csv"

                    final_data.to_csv(filename, index=False)

                    print(f"✅ Total rows: {len(final_data)}")
                    print(f"✅ File created: {filename}")
                    print("=" * 70)

                    # Display preview
                    display(HTML("<h3 style='color: #10b981; margin-top: 20px;'>✅ FULL Analysis Complete!</h3>"))
                    display(HTML(f"<p style='color: #64748b;'>📁 File: <strong>{filename}</strong></p>"))

                    display(HTML("<h4 style='color: #3b82f6;'>📋 Data Preview (First 10 rows):</h4>"))
                    display(final_data.head(10))

                    # Download file
                    files.download(filename)

                    display(HTML(f"""
                    <div style='background: #d1fae5; padding: 20px; border-radius: 10px;
                                border-left: 5px solid #10b981; margin-top: 20px;'>
                        <h3 style='color: #065f46; margin-top: 0;'>✅ Success!</h3>
                        <p style='color: #047857; margin: 0;'>
                            <strong>{filename}</strong> has been downloaded!<br>
                            Total rows: {len(final_data)} | Groups: {len(all_groups)} | Stocks: {len(unique_stocks)} | Period: {period} days
                        </p>
                    </div>
                    """))
                else:
                    display(HTML("""
                    <div style='background: #fee2e2; padding: 20px; border-radius: 10px;
                                border-left: 5px solid #ef4444; margin-top: 20px;'>
                        <h3 style='color: #991b1b; margin-top: 0;'>❌ Error</h3>
                        <p style='color: #dc2626; margin: 0;'>
                            No data was fetched. Please check tickers and try again.
                        </p>
                    </div>
                    """))

            except Exception as e:
                print("\n" + "=" * 70)
                print(f"CRITICAL ERROR: {str(e)}")
                print("=" * 70)
                import traceback
                traceback.print_exc()

                display(HTML(f"""
                <div style='background: #fee2e2; padding: 20px; border-radius: 10px;
                            border-left: 5px solid #ef4444; margin-top: 20px;'>
                    <h3 style='color: #991b1b; margin-top: 0;'>❌ Critical Error</h3>
                    <p style='color: #dc2626; margin: 0;'>
                        {str(e)}
                    </p>
                </div>
                """))

# Initialize the GUI
gui = StockAnalysisGUI()